# stps to gt the
1. <s>look into xarray</s>
2. <s>import engines </s>
3. <s>import aopy </s>
4. cast the trial lfp as a module
5. then build a frame
6. suply the frame wz frame data

In [1]:
import xarray
from engines_interfaces import PipeModule, PipeFrame
import aopy

print('import succeeded')

import succeeded


Q: can w automate t whole process o returning a fxn? 
t desgn pat is a factory pattern srt of th.
i.e a cls t builds mr cls, fun, wil lok into thz

In [ ]:
#imagine w h a fxn tht builds a module gvn a cur module

In [ ]:
# f now, just inherit the PipModule
dim_types = ('TRIAL', 'FREQ', "TIME")

"""
#TO-DO
-test the thing
-cast this into a frame

"""



#thz is jst a wraper f
class PipeTrialLfpLoader(PipeModule):
    """
    
    """
    def __init__():
        super().__init__()
        
        
        self.output_dims = ('TRIAL', 'TIME')
        
        self._params_name_ordered = ('monkeyDrive','trials',  )
        self._params_name_kwd = ('task_field', 'bn', 
                                'microdrive_name', 'file_type',
                               'verbose')
        
        


                            
    def run_mod():
        _data = aopy.datareader.read_trials_lfp(self._params_val_dict['monkeyDrive'], 
                                                     monkeyDrive['Trials'],
                                                    self._params_val_dict)
        #TO-DO if it is okay py c chk self._params_val_dict is treated lk **kwarg
        self._da_xarray = xr.DataArray(_data, dims = self.output_dims)
        
        
        
        
    
    

# adapt a fxn from a dif pkg
use multitaper meth as an eg.
nitime: time series anal in python
1. used in BMI3D, in fact, this is wher I fd abot it. 
can find the exact def in:
https://github.com/nipy/nitime/blob/6f95beabf337a16ed882067f2ea3d0b32399c901/nitime/algorithms/spectral.py

In [2]:
import nitime.algorithms as tsa

#thz is jst a wraper f

class PipeMultiTaperEstimater(PipeModule):
    """
    
    """
    def __init__():
        self.mod_name = self.__class__.__name__
        
        input_dims = ('TRIAL', 'TIME')
        
        #ASSUME: this is the output dims for tsa.
        self.output_dims = ('TRIAL', 'TIME')
        
        self._params_name_ordered = ('DATA')
        self._params_name_kwd = ('Fs', 'Nw')
        self._params_val_dict = dict()
        
    
    def run_mod():
        
        #for mor detail go to nitime.algorithms.spectral
        
        freqs, psd_est, var_or_nu = tsa.multi_taper_psd(cont_samples = self._params_val_dict['DATA'],
                                                        **self._params_val_dict)
        
        
        self._da_xarray = xr.DataArray(psd_est, dims = self.output_dims = ('TRIAL', 'TIME','FREQ'))

# t Vain atmp t lnk ld frame A mttaper

assume they are welrtn A tsted, whch thy n

In [ ]:
#ASSUM PipeTrialLfpLoader A PipeMultiTaperEstimater h b blt to t spec
#ASSUM: PipeFrame h b blt, unver of crse

trial_loader = PipeTrialLfpLoader()
mtf_estimater = PipeMultiTaperEstimater()

#now, bld two frames
#build t estimater frame 1st
"""
mtf_parameters = {
    'Fs': 1000,
    'NW': np.arange(1,10),#normalized  half-bandwidth
    'BW':np.arange(10,20) #sampl rela bandwidth
}
"""

mtf_parameters = {
    'Fs': 1000,
    'NW': np.arange(1,10),#normalized  half-bandwidth
    'BW':np.arange(10,20) #sampl rela bandwidth
}

# this is a dict of dictionaries
module_parameters = {
    mtf_estimater.mod_name:  mtf_parameters
}


mtf_estimaters_frame = PipeFrame(modules = [mtf_estimater], module_parameters)

#INS, 
#d t same th. for ld
#aft tht, run check compat 
                                